Automatically delint notebooks on save?



In [1]:
// TODO: display a tree of require dependencies and make sure it all still compiles properly
var importer = require('../Core');
var fs = require('fs');
var path = require('path');
var {ncp} = require('ncp');
var {execCmd, diffTwoTexts} = importer.import([
    'spawn child process',
    'diff code blocks',
]);
var cell = importer.interpret('diff code blocks');
ncp.limit = 16;

var webstormWin = 'C:\\Program Files\\JetBrains\\WebStorm 2017.1.1\\bin\\format.bat';
var webstormMac = '/Applications/WebStorm.app/Contents/bin/format.sh';
var webstorm = process.platform === 'win32' ? webstormWin : webstormMac;

// TODO: how to force CLI to use project based settings instead of user settings?
var idea = path.resolve('../.idea');

function delint(project) {
    return new Promise((resolve, reject) => {
        // output .idea to set the framework preferences?
        return ncp(idea, project + '/.idea', function (err) {
            if (err) {
                return reject(err);
            }
            resolve();
        });
    })
    // TODO: make this a test script for exporting all notebooks on file change watcher
        .then(() => {
            // I think I overwrote the default, some way to env variables for this?
            const cmd = '"' + webstorm + '" "' + project + '/"';
            var cwd = process.cwd();
            if (!fs.existsSync(project)) {
                fs.mkdirSync(project);
            }
            return execCmd(cmd, {cwd: project})
        })
};
module.exports = delint;



[Function: delint]

test de-linting service?


In [5]:
var importer = require('../Core');
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = path.resolve('../Utilities/.modules');

// just do it in load mode
if(typeof describe === 'undefined') {
    var it = ((l, func) => func());
    var beforeEach = ((func) => func());
    var describe = ((l, func) => func());
//    var describe = (() => {});
}

describe('de-linting service', () => {
    it('should de-lint a file', (done) => {
        const r = importer.interpret('diff code blocks');
        assert(r.filename.indexOf('diff.ipynb') > -1);
        delint(project).then(r => {
            console.log(r);
            done();
        })
        .then(r => {
            console.log(r);
            var filename = path.join(project, 'Utilities/diff/cell-' + cell.to + '.js');
            return diffTwoTexts(cell.code, fs.readFileSync(filename).toString());
        })
        .catch(r => console.log(r));
    }).timeout(10000)
})



AssertionError [ERR_ASSERTION]: false == true

Autofix some linting errors using bslint?

